In [ ]:
import cv2
import numpy as np


class PanoramaSticher:

    def __init__(self, path_1, path_2):

        self.img1 = cv2.imread(path_1)
        self.img_gray1 = cv2.cvtColor(self.img1, cv2.COLOR_BGR2GRAY)

        self.img2 = cv2.imread(path_2)
        self.img_gray2 = cv2.cvtColor(self.img2, cv2.COLOR_BGR2GRAY)

    def make_output(self):

        # Detect keypoints and features for both images
        kpsA, featuresA = cv2.SIFT_create().detectAndCompute(self.img_gray1, None)
        kpsB, featuresB = cv2.SIFT_create().detectAndCompute(self.img_gray2, None)

        # Match keypoints between the two images
        matches = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False).knnMatch(featuresA, featuresB, 2)

        # Select only good matches
        good_matches = [m for m, n in matches if m.distance < n.distance * 0.75]

        # Convert keypoints to numpy arrays
        kpsA = np.float32([kp.pt for kp in kpsA])
        kpsB = np.float32([kp.pt for kp in kpsB])

        # Get the corresponding points for the good matches
        ptsA = np.float32([kpsA[m.queryIdx] for m in good_matches])
        ptsB = np.float32([kpsB[m.trainIdx] for m in good_matches])

        # Compute homography matrix using RANSAC
        (H, status) = cv2.findHomography(ptsA, ptsB, cv2.RANSAC, 5)

        # Stitch the images together to create the panorama
        result = cv2.warpPerspective(self.img1, H, (self.img1.shape[1] + self.img2.shape[1], self.img2.shape[0]))
        result[0:self.img2.shape[0], 0:self.img2.shape[1]] = self.img2

        # Save the output image
        cv2.imwrite('Output.jpg', result)


PS = PanoramaSticher('1.jpg', '2.jpg')
PS.make_output()